In [123]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve

# 1. Problem Specification

Refer to Paper ["Dynamic Optimization Itegrating Modifier Adaptation Using Transient Measurements"](https://www.sciencedirect.com/science/article/pii/S0098135421000600) Section 5 "Williams Otto reactor case study"

## A. Static Plant

A plant undergoes reactions:
- There are 3 reactions
- A,B are the reactants
- C is a intermediate component
- P is a desired product
- E,G are the by-products

In [124]:
def cstr_p(x,u):

    #__Inputs__
    F_B = u[0] # Flowrate of component B
    T_r = u[1] # Temperature of the reactor

    #__States__
    X_A = x[0] # Concentration of component A
    X_B = x[1]
    X_C = x[2]
    X_P = x[3]
    X_E = x[4]
    X_G = x[5]

    #__Process Parameters__
    k10 = 9.9594*10**6
    k20 = 8.66124*10**9
    k30 = 1.6047*10e13
    Ea1 = 6666.7
    Ea2 = 8333.3
    Ea3 = 11111
    X_A0 = 10
    X_B0 = 10
    V_R = 2105
    F_A = 112.35
    F_R = F_A + F_B

    #__Reactions__
    k1 = k10*np.exp(-Ea1/(T_r+273.15))
    k2 = k20*np.exp(-Ea2/(T_r+273.15))
    k3 = k30*np.exp(-Ea3/(T_r+273.15))

    r1 = k1*X_A*X_B
    r2 = k2*X_B*X_C
    r3 = k3*X_P*X_C

    #__Equations__
    eq = np.zeros(6)
    eq[0] = F_A*X_A0 - F_R*X_A - V_R*r1 
    eq[1] = F_B*X_B0 - F_R*X_B - V_R*r1 - V_R*r2 
    eq[2] = -F_R*X_C + V_R*r1 - V_R*r2 - V_R*r3
    eq[3] = -F_R*X_P + V_R*r2 - V_R*r3
    eq[4] = -F_R*X_E + V_R*r2
    eq[5] = -F_R*X_G + V_R*r3

    return eq

## B. Plant Model

The model have similar but not the same model:
- There are 3 reactions
- A,B are reactants
- P is product 
- G is a byproduct
- There is no intermidiate component C

The model has parameters:
- n is a model parameter for reaction rate constant coefficient
- v is a model parameter for activation energy

In [125]:
def cstr_m(x,u,theta):

    #__Inputs__
    F_B = u[0] # Flowrate of component B
    T_r = u[1] # Temperature of the reactor

    #__States__
    X_A = x[0] # Concentration of component A
    X_B = x[1]
    X_P = x[2]
    X_E = x[3]
    X_G = x[4]

    #__Hyperparameters__
    n1 = theta[0]
    n2 = theta[1]
    v1 = theta[2]
    v2 = theta[3]

    #__Process Parameters__
    X_A0 = 10
    X_B0 = 10
    V_R = 2105
    F_A = 112.35
    F_R = F_A + F_B

    #__Reactions__
    k1 = n1*np.exp(-v1/(T_r+273.15))
    k2 = n2*np.exp(-v2/(T_r+273.15))

    r1 = k1*X_A*X_B**2
    r2 = k2*X_A*X_B*X_P

    #__Equations__
    eq = np.zeros(5)
    eq[0] = F_A*X_A0 - F_R*X_A - V_R*r1 - V_R*r2
    eq[1] = F_B*X_B0 - F_R*X_B - 2*V_R*r1 - V_R*r2 
    eq[2] = -F_R*X_P + V_R*r1 - V_R*r2
    eq[3] = -F_R*X_E + V_R*r1
    eq[4] = -F_R*X_G + V_R*r2

    return eq

In [126]:
# Sample Calculation
x0 = [5,5,5,5,5] # Initial Guess
u = np.array([100,30]) # Parameters F_B and T_r
theta = np.array([1.3134e8,2.586e13,8077.6,12438.5]) # Hyperparameters k0 and Ea
x = fsolve(cstr_m, x0, args=(u,theta),xtol=1e-6) # scipy fsolve to solve equations

print(f"The concetrations of components are: {x}")

The concetrations of components are: [4.99330771e+00 4.11657306e+00 2.92809515e-01 2.95147651e-01
 2.33813582e-03]


## C. Cost Function 

The cost function is based on quantities that are on outlet of the CSTR. Therefore, the equations on CSTR are solved before we calculate the cost function. The equations on cstr are solved by scipy fsolve.

In [127]:
def cost_function(u,theta):
       
       #__Constants__
       P_A = 7.623
       P_B = 11.434
       P_P = 114.338
       P_E = 5.184
       F_A = 112.35
       X_A0 = 10
       X_B0 = 10

       #__Process Parameters__
       F_B = u[0]
       F_R = F_A + F_B
       
       #__Solve Equations on cstr.m__
       x0 = [4,4,4,4,4]
       x = fsolve(cstr_m, x0, args=(u,theta),xtol=1e-6)
       X_P = x[2]
       X_E = x[3]
       
       #__Calculate cost function__
       cost = -(F_R*(X_P*P_P + X_E*P_E) - F_A*X_A0*P_A - F_B*X_B0*P_B) # -ve sign to maximize the cost function using scipy minimize
       
       return cost 

In [128]:
# Sample Calculation
u = np.array([292.24204707,78.408519])
theta = np.array([1.3134*1e8,2.586*1e13,8077.6,12438.5])
cost = cost_function(u,theta)
print(f"cost: {cost}")

cost: -11312.209156316734


# 2. Model-parameter Adaptation



## A. Optimization on cost function

Optimization is done with Simplex Method. Following is a sample calculation of optimization given certain parameters

In [129]:
def cost_optimize(u0,theta):
    cons = (
        {'type': 'ineq', 'fun': lambda x: -(x[0]-1.2)},
        {'type': 'ineq', 'fun': lambda x: -(x[4]-0.5)}
    )

    bnds = ((180,360),(75,100))

    res = minimize(cost_function, u0, 
                method='nelder-mead', 
                bounds = bnds,
                constraints= cons,
                args=(theta))

    return [res.x,res.fun]

In [130]:
# Sample Calculation
u0 = np.array([350,82])
theta = np.array([1.3134*1e8,2.586*1e13,8077.6,12438.5])
optimal_input, optimal_output = cost_optimize(u0,theta)
print(f"optimal input: {optimal_input}")
print(f"optimal output: {optimal_output}")

optimal input: [292.24217012  78.40851052]
optimal output: -11312.209156316618


## B. Model Adaptation
We try to minimize difference between output of an actual plant and output of a model. The difference is measured by root mean squared error, which is used for cost function to be minimized by simplex algorithm


In [131]:
# root mean squared error bewteen actual plant and output of a model
def difference(theta,u):
    x0_p = [4,4,4,4,4,4]
    x0 = [4,4,4,4,4]
    x_p = fsolve(cstr_p, x0_p, args=(u), xtol=1e-6)
    x = fsolve(cstr_m, x0, args=(u,theta), xtol=1e-6)

    x_p = np.delete(x_p,2)
    
    squared_root_mean_error = np.sqrt(np.mean((x_p-x)**2))

    return squared_root_mean_error



# Input derived from optimization (maximization) of a cost function
u = np.array([292.24217012, 78.40851052])

def adaptation(theta0,u):
    bnds = ((0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf))
    theta0 = np.array([1.3134e8, 2.586e13, 8077.6, 12438.5])  # Four elements in theta0

    res = minimize(difference, theta0,
                method='nelder-mead',
                bounds=bnds,
                args=(u))
    
    return res.x


## C. Overall Algorithm

NEED TO BE FIXED

In [132]:
# Initial Guess
u = np.array([350,82])
theta = np.array([1.3134*1e8,2.586*1e13,8077.6,12438.5])
cost_list = [] # record the change in cost along the iterations
for i in range(10):
    
    u,cost = cost_optimize(u,theta)
    cost_list.append(cost)
    theta = adaptation(theta,u)

print(cost_list)
print(u)
print(theta)
    
    


[-11312.209156316618, 15760.091346683399, 16196.580779809436, 16196.580779809436, 16196.580779809436, 16196.580779809436, 16196.580779809436, 16196.580779809436, 16196.580779809436, 16196.580779809436]
[180.  75.]
[1.34086827e+08 2.65998670e+13 8.16656493e+03 1.17633548e+04]
